In [1]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import when, col

In [2]:
# Initialize a Spark session
spark = SparkSession.builder.appName("DataTransformation").getOrCreate()

23/09/05 14:37:17 WARN Utils: Your hostname, ubuntu-Lenovo-Legion-5-15ARH05 resolves to a loopback address: 127.0.1.1; using 172.16.5.112 instead (on interface wlp4s0)
23/09/05 14:37:17 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
23/09/05 14:37:18 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
23/09/05 14:37:19 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


**Reading csv**


In [3]:
#reading csv file
listing_df = spark.read.csv('cleaned_data/clean_listing_csv/', header=True, inferSchema=True)
review_df = spark.read.csv('cleaned_data/clean_reviews_csv/', header=True, inferSchema=True)
calender_df = spark.read.csv('cleaned_data/clean_calendar_csv/', header=True, inferSchema=True)


